In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import auto_vec_filter_helper.auto_vec_filter_data as avf_data
import seaborn as sns
import math

def subset_data_size(df, size_column):
    return df[df[size_column].isin([4*1024*1024, 128*1024*1024, 1*1024*1024*1024])]

# Aggregate

## Scalar

In [2]:
df = pd.read_csv("aggregate/old/results.tsv", sep='\t')
df_mean = df.groupby(["device","arch-flags","function","variant","SIMD-elementCount", "byte_size","element_size"])["time_ns"].mean().reset_index()
df_mean["tput GiB/s"] = (df_mean["byte_size"]/1024/1024/1024)/(df_mean["time_ns"]*1e-9)
df_mean["beauty_bytes"] = df_mean["byte_size"].apply(lambda x: avf_data.beautify_data_size(x))
df_mean["beauty_bytes_no_ending"] = df_mean["byte_size"].apply(lambda x: avf_data.beautify_data_size(x, False))
df_mean = df_mean[ (df_mean["arch-flags"] == "FPGA") ]
df_mean["color_id"]=df_mean["SIMD-elementCount"].apply(lambda x: int(math.log2(x)))
df_mean["SIMD-elementCount"]=df_mean["SIMD-elementCount"].apply(lambda x: int(x))
sns.set(style="whitegrid", rc={'grid.linestyle': '--'})
g = sns.barplot(data=df_mean[ df_mean["element_size"] == 8 ],  x="beauty_bytes_no_ending", y="tput GiB/s", hue="SIMD-elementCount", palette=sns.color_palette("muted"))
#rotate x labels by 30 degrees
for item in g.get_xticklabels():
    item.set_rotation(30)
g.set_ylim(0,3)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlabel("Processed Data Size [MiB]", fontsize=22)
plt.ylabel("Throughput [GiB/s]", fontsize=22)
plt.legend().remove()
plt.tight_layout()
plt.savefig("submission/aggregate_throughput.pdf")
plt.close()

## Autovec

In [3]:
df = pd.read_csv("aggregate/aggregate_results.tsv", sep='\t')
df = subset_data_size(df, "byte_size")
df_mean = df.groupby(["device","compiler","version","arch-flags","function","variant","SIMD-elementCount","byte_size","element_size","element_count"])["time_ns"].mean().reset_index()
df_mean["tput GiB/s"] = (df_mean["byte_size"]/1024/1024/1024)/(df_mean["time_ns"]*1e-9)
df_mean["beauty_bytes"] = df_mean["byte_size"].apply(lambda x: avf_data.beautify_data_size(x))
df_mean["beauty_bytes_no_ending"] = df_mean["byte_size"].apply(lambda x: avf_data.beautify_data_size(x, False))
df_mean = df_mean[ (df_mean["arch-flags"] == "FPGA") ]
df_mean["color_id"]=df_mean["SIMD-elementCount"].apply(lambda x: int(math.log2(x)))
df_mean["SIMD-elementCount"]=df_mean["SIMD-elementCount"].apply(lambda x: int(x))
sns.set(style="whitegrid", rc={'grid.linestyle': '--'})
cm = sns.color_palette('coolwarm', as_cmap=True)
g = sns.barplot(data=df_mean,  x="beauty_bytes_no_ending", y="tput GiB/s", hue="SIMD-elementCount", palette=sns.color_palette("muted"),)
g.legend(title="Vector Length", loc='upper center', bbox_to_anchor=(0.5, -0.3), ncol=8)
plt.setp(g.get_legend().get_texts(), fontsize='18') 
plt.setp(g.get_legend().get_title(), fontsize='20')
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlabel("Processed Data Size [MiB]", fontsize=22)
plt.ylabel("Throughput [GiB/s]", fontsize=22)
plt.tight_layout()
plt.savefig("submission/aggregate_autovec_all_throughput.pdf", bbox_inches='tight', pad_inches = 0)
plt.savefig("submission/aggregate_autovec_all_throughput.png", bbox_inches='tight', pad_inches = 0)
plt.close()
sns.set(style="whitegrid", rc={'grid.linestyle': '--'})
cm = sns.color_palette('coolwarm', as_cmap=True)
g = sns.barplot(data=df_mean[df_mean["SIMD-elementCount"]<16],  x="beauty_bytes_no_ending", y="tput GiB/s", hue="SIMD-elementCount", palette=sns.color_palette("muted"))
g.legend(title="Vector Length", loc='upper center', bbox_to_anchor=(0.5, -0.3), ncol=8)
plt.setp(g.get_legend().get_texts(), fontsize='18') 
plt.setp(g.get_legend().get_title(), fontsize='20')
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlabel("Processed Data Size [MiB]", fontsize=22)
plt.ylabel("Throughput [GiB/s]", fontsize=22)
plt.tight_layout()
plt.savefig("submission/aggregate_autovec_128_to_256_throughput.pdf", bbox_inches='tight', pad_inches = 0)
plt.savefig("submission/aggregate_autovec_128_to_256_throughput.png", bbox_inches='tight', pad_inches = 0)
plt.close()

# Modulo

In [7]:
df = pd.read_csv("modulo_results/benchmark_results_modulo.tsv", sep='\t')
#df = subset_data_size(df, "datasize")
df = df[(df["flavor"] == "percent operator") ]
df["beauty_bytes"] = df["datasize"].apply(lambda x: avf_data.beautify_data_size(x))
df["beauty_bytes_no_ending"] = df["datasize"].apply(lambda x: avf_data.beautify_data_size(x, False))
sns.set(style="whitegrid", rc={'grid.linestyle': '--'})
cm = sns.color_palette('coolwarm', as_cmap=True)
g = sns.barplot(data=df,  x="beauty_bytes_no_ending", y="throughput [GiB/s]", hue=[1]*len(df), palette=sns.color_palette("muted"))
#rotate x labels by 30 degrees
for item in g.get_xticklabels():
    item.set_rotation(30)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlabel("Processed Data Size [MiB]", fontsize=22)
plt.ylabel("Throughput [GiB/s]", fontsize=22)
plt.legend().remove()
plt.tight_layout()
plt.savefig("submission/modulo_aggregate.pdf")
plt.close()

# Conflict Detection

In [8]:
df = pd.read_csv("conflict/benchmark_results_conflict_aggregate.tsv", sep='\t')
#df = subset_data_size(df, "datasize")
df["tput GiB/s"] = (df["datasize"]/(1024*1024*1024)) / (df["execution_times"] / 1000000)
df["beauty_bytes"] = df["datasize"].apply(lambda x: avf_data.beautify_data_size(x))
df["beauty_bytes_no_ending"] = df["datasize"].apply(lambda x: avf_data.beautify_data_size(x, False))
sns.set(style="whitegrid", rc={'grid.linestyle': '--'})
cm = sns.color_palette('coolwarm', as_cmap=True)
g = sns.barplot(data=df,  x="beauty_bytes_no_ending", y="tput GiB/s", hue=[1]*len(df), palette=sns.color_palette("muted"))
#rotate x labels by 30 degrees
for item in g.get_xticklabels():
    item.set_rotation(30)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlabel("Processed Data Size [MiB]", fontsize=22)
plt.ylabel("Throughput [GiB/s]", fontsize=22)
plt.legend().remove()
plt.tight_layout()
plt.savefig("submission/cd_aggregate.pdf")
plt.close()


# Count Leading Zeroes

In [9]:
df = pd.read_csv("clz_results/benchmark_results_clz_aggregate.tsv", sep='\t')
#df = subset_data_size(df, "datasize")
df = df[df["flavor"]=="loop-based"]
df["beauty_bytes"] = df["datasize"].apply(lambda x: avf_data.beautify_data_size(x))
df["beauty_bytes_no_ending"] = df["datasize"].apply(lambda x: avf_data.beautify_data_size(x, False))
sns.set(style="whitegrid", rc={'grid.linestyle': '--'})
cm = sns.color_palette('coolwarm', as_cmap=True)
g = sns.barplot(data=df,  x="beauty_bytes_no_ending", y="throughput [GiB/s]", hue=[1]*len(df), palette=sns.color_palette("muted"))
#rotate x labels by 30 degrees
for item in g.get_xticklabels():
    item.set_rotation(30)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlabel("Processed Data Size [MiB]", fontsize=22)
plt.ylabel("Throughput [GiB/s]", fontsize=22)
plt.legend().remove()
plt.tight_layout()
plt.savefig("submission/clz_aggregate.pdf")
plt.close()